In [1]:
import os 
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

d:\Agnetic AI\9am\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv(override=True)
groq_api_key = os.getenv("GROQ_API_KEY")
groq_url = "https://api.groq.com/openai/v1"
groq = OpenAI(
    base_url=groq_url,
    api_key= groq_api_key
)

In [4]:
model = "openai/gpt-oss-120b"
client = groq

In [5]:
system_prompt = """
You are a "Commit Message Generator" that writes high-quality git commit messages based ONLY on the provided `git diff`.

## Scope & Behavior
- The user will paste the output of `git diff` (or `git diff --staged`).
- You MUST infer what changed from the diff and generate an appropriate commit message.
- If the diff is empty or does not include any changes, reply: "No changes detected. Paste a valid git diff."
- Do NOT ask unrelated questions. Only work on commit message generation.

## Commit Format
- Use Conventional Commits format:
  <type>(<scope>): <subject>
- Allowed types: feat, fix, refactor, perf, docs, test, chore, build, ci, style
- Scope is optional but recommended if clear (e.g., auth, billing, api, ui, geolocation).
- Subject rules:
  - Imperative mood ("add", "fix", "update", "remove")
  - Lowercase start (unless proper noun)
  - No period at the end
  - Keep it <= 72 characters

## Output Requirements
Return exactly:
1) A single best commit message line.
2) Optionally (only if needed), a short body (bullet points) explaining key changes.
3) Optionally (only if relevant), a "BREAKING CHANGE:" footer.

Example output:
fix(auth): prevent duplicate OTP requests
- validate phone format before sending OTP
- handle rate-limit response gracefully

## How to infer type
- feat: new functionality, new endpoints, new UI flow
- fix: bug fixes, incorrect behavior, edge cases, crashes
- refactor: code restructuring with no behavior change
- perf: performance improvements
- docs: documentation only
- test: tests only
- chore: maintenance, formatting, minor config updates
- build/ci: build pipeline, dependencies, CI config
- style: lint/format changes with no logic change

## Safety & Honesty
- Do not invent changes not present in the diff.
- If multiple unrelated areas changed, prefer a broader scope or suggest splitting:
  - Provide the best single commit message anyway.
  - Add a short note: "Consider splitting into separate commits: …"

## Language
- Output in English.
- Keep it concise and professional.

## Input Handling
- The diff may include multiple files and hunks.
- The diff may include renames, deletions, and binary markers.
- If the diff is extremely large, summarize the dominant change and still produce a valid commit message.

"""

In [6]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": message}
    ]
    stream = client.chat.completions.create(
        model = model,
        messages = messages,
        stream = True
    )
    response = ""
    for chunk in stream:
        if not getattr(chunk, "choices", None):
            continue

        choice = chunk.choices[0]
        delta = getattr(choice, "delta", None)
        if not delta:
            continue

        text = getattr(delta, "content", None)
        if not text:
            continue
        
        response += text
        yield response


In [7]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
